In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
import pandas as pd

In [ ]:
import GEOparse
gse = GEOparse.get_GEO("GSE62564") #GSE62564 has EFS data
gse2 = GEOparse.get_GEO("GSE49711") #GSE49711 has microarray data

In [7]:
#Load into dataframes
clinical = gse.phenotype_data
clinical2 = gse2.phenotype_data

In [8]:
#Formatting and merginf dataframes
clinical = clinical.reset_index()
clinical2 = clinical2.reset_index()
clinical = pd.concat([clinical, clinical2], axis=1)

In [9]:
#Get just the columns I'm interested in and rename them
reduced_clinical = clinical[['index', 'title', 'characteristics_ch1.1.Sex', 'characteristics_ch1.2.age', 'characteristics_ch1.3.efs day', 'characteristics_ch1.4.efs bin', 'characteristics_ch1.4.mycn status']]

In [11]:
#Rename columns and drop the duplicates
reduced_clinical.columns = ['GSM', 'GSM2', 'SampID2', 'SampID', 'Sex', 'Age', 'EFS Time', 'EFS Bin', 'MYCN Amplification']
reduced_clinical = reduced_clinical.drop(['GSM2', 'SampID2'], axis=1)

In [12]:
#Load microarray data file
microarray_data = pd.read_csv('/gdrive/My Drive/project/GSE49711_SEQC_NB_MAV_G_log2.csv', sep='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
#Format microarray dataframe so it. can be merged with clinical data
microarray_data = microarray_data.transpose()
microarray_data.columns = microarray_data.iloc[0]
microarray_data = microarray_data.drop(['#Gene'])
#Genes I'm interested in
genes = microarray_data[['TP53', 'NTRK1', 'PTPN6', 'ACE2', 'TMPRSS2', 'CXCL10', 'AGTR1']]
genes = genes[1:]
genes = genes.reset_index()
genes = genes.rename(columns={"index": "SampID"})

In [15]:
#Merge clinical data and microarray data
all = pd.merge(reduced_clinical, genes, on='SampID')

In [17]:
#Calculate z-scores
from scipy.stats import zscore
all['TP53_z'] = zscore(all['TP53'].astype(float))
all['NTRK1_z'] = zscore(all['NTRK1'].astype(float))
all['PTPN6_z'] = zscore(all['PTPN6'].astype(float))
all['ACE2_z'] = zscore(all['ACE2'].astype(float))
all['TMPRSS2_z'] = zscore(all['TMPRSS2'].astype(float))
all['CXCL10_z'] = zscore(all['CXCL10'].astype(float))
all['AGTR1_z'] = zscore(all['AGTR1'].astype(float))

In [18]:
all

,GSM,SampID,Sex,Age,EFS Time,EFS Bin,MYCN Amplification,TP53,NTRK1,PTPN6,ACE2,TMPRSS2,CXCL10,AGTR1,TP53_z,NTRK1_z,PTPN6_z,ACE2_z,TMPRSS2_z,CXCL10_z,AGTR1_z
0,GSM1528894,SEQC_NB001,M,987,593,1,0,13.97,15.35,15.36,3.99,4.21,13.26,8.25,-0.177807,-0.134065,2.205290,-0.128659,0.405994,0.963559,-0.569730
1,GSM1528895,SEQC_NB002,M,1808,2016,1,0,13.61,13.97,13.11,7,0,12.54,7.75,-0.643984,-0.624117,0.308133,0.936732,-1.091646,0.697643,-0.759969
2,GSM1528896,SEQC_NB003,F,625,840,1,1,15.59,9.27,12.45,7.44,4.44,0,8.86,1.919990,-2.293134,-0.248366,1.092470,0.487813,-3.933721,-0.337639
3,GSM1528897,SEQC_NB004,F,335,2046,0,1,14.78,12.99,12.32,7.62,4.15,11.37,12.34,0.871092,-0.972125,-0.357979,1.156181,0.384650,0.265530,0.986421
4,GSM1528898,SEQC_NB005,F,536,212,1,1,14.85,9.11,11.8,4.43,0,9.41,7.56,0.961737,-2.349952,-0.796433,0.027079,-1.091646,-0.458351,-0.832259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,GSM1529389,SEQC_NB494,M,56,4828,0,0,13.94,17.48,12.7,5.3,5.06,10.65,11.96,-0.216655,0.622320,-0.037571,0.335016,0.708368,-0.000386,0.841840
494,GSM1529390,SEQC_NB495,M,163,2467,0,0,14.02,18.16,13.03,7.19,5.87,14.68,10.15,-0.113060,0.863795,0.240679,1.003982,0.996512,1.488003,0.153176
495,GSM1529391,SEQC_NB496,M,132,105,1,0,14.02,15.84,11.27,6.45,5.2,9.38,10.87,-0.113060,0.039939,-1.243319,0.742059,0.758170,-0.469431,0.427120
496,GSM1529392,SEQC_NB497,F,379,4883,0,0,14.13,16.65,12.91,7.21,4.81,11.92,8.2,0.029383,0.327578,0.139497,1.011061,0.619434,0.468660,-0.588754
